In [1]:
import numpy as np
import numba as nb

import ice_funcs as icef

## Todo:
- ***Make sure that diffusion works!!***
- ***Figure out if I actually need to hold on to timings... and if yes... how ?***
    - Is the timing array even needed &&&**[do not delete, just comment out]**&&&
- Make relaxSaturationUtilities fully OOP


## Backburner-type-beat:
- make sure everything is numbafied
- Continue commenting in `ice_funcs.py`

## Improvements
- Monopole matching
- correct geometry factors $G_b$ and $H_b$
- Causal timings?

In [2]:
# CONSTANTS

L = 101
W = (L+1)//2
INITIAL_SAT = 1

In [3]:
gu = icef.GeneralUtilities(101) # general utilities

pu = icef.PhysicsUtilities(0.01, 10) # physics utilities



In [4]:
sru = icef.SaturationRelaxationUtilities(101, pu, 200)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m- Resolution failure for literal arguments:
[1mFailed in nopython mode pipeline (step: native lowering)
LLVM IR parsing error
<string>:467:140: error: base element of getelementptr must be sized
  %".1702" = getelementptr inbounds {i32, i32, %"deferred.140471490431008.data", i32, {i8*, i8*, i64, i64, float*, [3 x i64], [3 x i64]}}, {i32, i32, %"deferred.140471490431008.data", i32, {i8*, i8*, i64, i64, float*, [3 x i64], [3 x i64]}}* %".1701", i32 0, i32 1
                                                                                                                                           ^
[0m
[0m[1m- Resolution failure for non-literal arguments:
[1mNone[0m
[0m[0m
[0m[1mDuring: resolving callee type: BoundFunction((<class 'numba.core.types.misc.ClassInstanceType'>, '_construct_default_diffusion_rules') for insta

In [ ]:
neighbor_array_test = gu.neighbor_array
diffusion_rules_test = sru.diffusion_rules

In [ ]:
default_ice_map = np.full((101, 101), False)
default_ice_map[-1,0] = True
default_ice_map[-2,0] = True
default_ice_map[-3,0] = True
default_ice_map[-3,1] = True

# %timeit test_boundary_map
boundary_map_test = gu.construct_boundary_map(default_ice_map)
print(boundary_map_test[-5:,:5])

sat_map_test = np.full((101,101), 1, dtype=np.float32)

normal_cells_test, boundary_cells_test = gu._distinguish_cells(default_ice_map, boundary_map_test)

opp_array_test = sru._construct_opp_array(boundary_cells_test, default_ice_map, neighbor_array_test)

[[0 0 0 0 0]
 [1 2 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [ ]:
# pu.apply_boundary_condition(2, 2, sat_map_test, 0.5, boundary_map_test)

sat_map_test = np.full((101,101), 1, dtype=np.float32)

# for i in range(50):
#     sru._execute_relaxation_step(sat_map_test, normal_cells_test, boundary_cells_test, opp_array_test, neighbor_array_test, boundary_map_test)

new_thang = sru.diffuse_to_convergence(sat_map_test, 200, 0.001, normal_cells_test, boundary_cells_test, default_ice_map, boundary_map_test, neighbor_array_test)
print(new_thang[1])
print(new_thang[2])
print(new_thang[0][-10:,:5])

True
4
[[1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [0.9999831  0.9999212  0.9998375  0.99979866 1.        ]
 [0.99968517 0.99934995 0.9991115  1.         1.        ]
 [0.9984669  0.9972105  0.9965122  1.         1.        ]
 [0.9952073  0.991714   1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]]
